# 把框选出的单词单独保存出来

In [49]:
import cv2
import os
import numpy as np
from PIL import Image, ImageOps

# 把黑像素点变成白像素点，白像素点变成黑像素点

In [50]:
def crop_black_background(image_path):
    """Crop out the black background from an image and return the processed image."""
    try:
        # Open the image
        image = Image.open(image_path)
        
        # Check if image has an alpha channel
        if image.mode == 'RGBA':
            r, g, b, a = image.split()
            rgb_image = Image.merge('RGB', (r, g, b))
        else:
            rgb_image = image.convert('RGB')

        # Invert the image
        inverted_image = ImageOps.invert(rgb_image)
        
        # Re-add the alpha channel if it was originally present
        if image.mode == 'RGBA':
            r2, g2, b2 = inverted_image.split()
            final_image = Image.merge('RGBA', (r2, g2, b2, a))
        else:
            final_image = inverted_image.convert('RGBA')
        
        return final_image

    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

def process_images_in_folder(folder_path):
    """Process all images in a folder, crop out the black background, and overwrite the original images."""
    # List all files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            # Construct full file path
            img_path = os.path.join(folder_path, file_name)
            
            # Process image
            cropped_image = crop_black_background(img_path)
            
            if cropped_image:
                # Save the processed image by overwriting the original image
                cropped_image.save(img_path)
                print(f"Processed and saved {img_path}")
            

In [51]:
folder_path = '/root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL'  # Path to the folder containing images

# Process all images in the folder
process_images_in_folder(folder_path)

Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL/CVL_1052.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL/CVL_0.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL/CVL_1053.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL/CVL_1.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL/CVL_1054.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL/CVL_10.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL/CVL_1055.png


Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL/CVL_100.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL/CVL_1057.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL/CVL_1000.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL/CVL_1056.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL/CVL_1001.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL/CVL_1058.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL/CVL_1002.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL/CVL_1059.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL/CVL_1003.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL/CVL_1069.png
P

In [52]:
folder_path = '/root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/IAM'  # Path to the folder containing images

# Process all images in the folder
process_images_in_folder(folder_path)

Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/IAM/IAM_1052.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/IAM/IAM_0.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/IAM/IAM_1053.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/IAM/IAM_1.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/IAM/IAM_1054.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/IAM/IAM_10.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/IAM/IAM_1055.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/IAM/IAM_100.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/IAM/IAM_1057.png
Processed and saved /root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/IAM/IAM_1000.png
Processed

# 把图片截出来

In [60]:
def extract_colored_boxes(image):
    # Convert the image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define the lower and upper bounds for black and white colors in HSV
    lower_black = np.array([0, 0, 0])
    upper_black = np.array([180, 255, 50])  # Black color range

    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 25, 255])  # White color range

    # Create masks for black and white colors
    black_mask = cv2.inRange(hsv, lower_black, upper_black)
    white_mask = cv2.inRange(hsv, lower_white, upper_white)

    # Create a mask for pixels that are neither black nor white
    not_black_mask = cv2.bitwise_not(black_mask)
    not_white_mask = cv2.bitwise_not(white_mask)
    colored_mask = cv2.bitwise_and(not_black_mask, not_white_mask)

    # Find contours in the colored mask
    contours, _ = cv2.findContours(colored_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours, colored_mask

def resize_image(image, size=(128, 128)):
    return cv2.resize(image, size)

def crop_words(input_folder, crop_folder):
    if not os.path.exists(crop_folder):
        os.makedirs(crop_folder)

    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    for img_file in image_files:
        img_path = os.path.join(input_folder, img_file)
        image = cv2.imread(img_path)
        
        contours, _ = extract_colored_boxes(image)
        
        # Create a subfolder for each image
        img_subfolder = os.path.join(crop_folder, os.path.splitext(img_file)[0])
        if not os.path.exists(img_subfolder):
            os.makedirs(img_subfolder)
        
        for i, cnt in enumerate(contours):
            x, y, w, h = cv2.boundingRect(cnt)
            word_crop = image[y:y+h, x:x+w]
            
            # Save cropped image with a unique name within the subfolder
            crop_path = os.path.join(img_subfolder, f"word_{i}.png")
            cv2.imwrite(crop_path, word_crop)

def resize_and_clean_words(crop_folder):
    if not os.path.exists(crop_folder):
        raise ValueError(f"The folder '{crop_folder}' does not exist.")

    for root, dirs, files in os.walk(crop_folder):
        for img_file in files:
            if img_file.endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(root, img_file)
                image = cv2.imread(img_path)
                
                # Resize the image
                resized_image = resize_image(image)
                
                # Save the resized image (overwrite the original cropped image)
                cv2.imwrite(img_path, resized_image)

In [65]:
#CVL
input_folder = "/root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL"
crop_folder = "/root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL_rec"

# Step 1: Crop words and save them to crop_folder
crop_words(input_folder, crop_folder)


In [66]:
# Step 2: Resize cropped images and remove red boxes
resize_and_clean_words(crop_folder)

In [67]:
#IAM
input_folder = "/root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/IAM"
crop_folder = "/root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/IAM_rec"

# Step 1: Crop words and save them to crop_folder
crop_words(input_folder, crop_folder)


In [68]:
# Step 2: Resize cropped images and remove red boxes
resize_and_clean_words(crop_folder)